<a href="https://colab.research.google.com/github/eksalailia/Big-Data/blob/main/JobsheetPyspark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [3]:
!wget -q https://dlcdn.apache.org/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz
!tar xf spark-3.2.1-bin-hadoop3.2.tgz

In [4]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.2.1-bin-hadoop3.2"


In [5]:
!pip install -q findspark

In [6]:
import findspark
findspark.init()

In [7]:
from pyspark.sql import SparkSession
spark = SparkSession.builder\
 .master("local")\
 .appName("Colab")\
 .config('spark.ui.port', '4050')\
 .getOrCreate()

In [8]:
!wget --continue https://github.com/dhanifudin/pyspark-demo -O sample_books.json

--2022-06-06 11:35:25--  https://github.com/dhanifudin/pyspark-demo
Resolving github.com (github.com)... 140.82.112.3
Connecting to github.com (github.com)|140.82.112.3|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘sample_books.json’

sample_books.json       [ <=>                ] 143.96K  --.-KB/s    in 0.007s  

2022-06-06 11:35:25 (19.7 MB/s) - ‘sample_books.json’ saved [147413]



In [11]:
df = spark.read.json("sample_books.json")

In [12]:
df.printSchema()

root
 |-- author: string (nullable = true)
 |-- edition: string (nullable = true)
 |-- price: double (nullable = true)
 |-- title: string (nullable = true)
 |-- year_written: long (nullable = true)



In [13]:
df.show(4,False)

+---------------+--------------+-----+----------------+------------+
|author         |edition       |price|title           |year_written|
+---------------+--------------+-----+----------------+------------+
|Austen, Jane   |Penguin       |18.2 |Northanger Abbey|1814        |
|Tolstoy, Leo   |Penguin       |12.7 |War and Peace   |1865        |
|Tolstoy, Leo   |Penguin       |13.5 |Anna Karenina   |1875        |
|Woolf, Virginia|Harcourt Brace|25.0 |Mrs. Dalloway   |1925        |
+---------------+--------------+-----+----------------+------------+
only showing top 4 rows



In [14]:
df.count()

13

In [15]:
df.select("title", "price", "year_written").show(5)

+----------------+-----+------------+
|           title|price|year_written|
+----------------+-----+------------+
|Northanger Abbey| 18.2|        1814|
|   War and Peace| 12.7|        1865|
|   Anna Karenina| 13.5|        1875|
|   Mrs. Dalloway| 25.0|        1925|
|       The Hours|12.35|        1999|
+----------------+-----+------------+
only showing top 5 rows



In [16]:
df_filtered = df.filter("year_written > 1950 AND price > 10 AND title IS NOT NULL")
df_filtered.select("title", "price", "year_written").show(50,False)

+-----------------------------+-----+------------+
|title                        |price|year_written|
+-----------------------------+-----+------------+
|The Hours                    |12.35|1999        |
|Harry Potter                 |19.95|2000        |
|One Hundred Years of Solitude|14.0 |1967        |
+-----------------------------+-----+------------+



In [18]:
from pyspark.sql.functions import max

# Find the costliest book
maxValue = df_filtered.agg(max("price")).collect()[0][0]
print("maxValue: ",maxValue)

df_filtered.select("title","price").filter(df.price == maxValue).show(20, False)

maxValue:  19.95
+------------+-----+
|title       |price|
+------------+-----+
|Harry Potter|19.95|
+------------+-----+



TUGAS !

In [19]:
df.show()

+--------------------+-----------------+-----+--------------------+------------+
|              author|          edition|price|               title|year_written|
+--------------------+-----------------+-----+--------------------+------------+
|        Austen, Jane|          Penguin| 18.2|    Northanger Abbey|        1814|
|        Tolstoy, Leo|          Penguin| 12.7|       War and Peace|        1865|
|        Tolstoy, Leo|          Penguin| 13.5|       Anna Karenina|        1875|
|     Woolf, Virginia|   Harcourt Brace| 25.0|       Mrs. Dalloway|        1925|
|Cunnningham, Michael|   Harcourt Brace|12.35|           The Hours|        1999|
|         Twain, Mark|          Penguin| 5.76|    Huckleberry Finn|        1865|
|    Dickens, Charles|     Random House| 5.75|         Bleak House|        1870|
|         Twain, Mark|     Random House| 7.75|          Tom Sawyer|        1862|
|     Woolf, Virginia|          Penguin| 29.0| A Room of One's Own|        1922|
|       Rowling, J.K.|   Har

* Tampilkan data buku dengan harga paling murah!





In [20]:
from pyspark.sql.functions import min

# Find the cheapest price

minValue = df.agg(min("price")).collect()[0][0]
print("minValue: ",minValue)

df.select("author", "edition", "price", "title", "year_written").filter(df.price == minValue).show(20, False)

minValue:  5.75
+----------------+------------+-----+-----------+------------+
|author          |edition     |price|title      |year_written|
+----------------+------------+-----+-----------+------------+
|Dickens, Charles|Random House|5.75 |Bleak House|1870        |
+----------------+------------+-----+-----------+------------+



* Tampilkan jumlah terbit buku dikategorikan setiap tahun ditulis!



In [27]:
import pyspark.sql.functions as f
from pyspark.sql.functions import desc

df.groupBy("year_written").count().select("year_written", f.col("count").
alias ("jml_terbit_setiap_tahun")).sort(desc("year_written")).show()

+------------+-----------------------+
|year_written|jml_terbit_setiap_tahun|
+------------+-----------------------+
|        2000|                      1|
|        1999|                      1|
|        1967|                      1|
|        1937|                      1|
|        1925|                      1|
|        1922|                      1|
|        1875|                      1|
|        1870|                      1|
|        1865|                      2|
|        1862|                      1|
|        1814|                      1|
|        1603|                      1|
+------------+-----------------------+



* Tampilkan data buku termahal tiap tahun penulisannya!

In [25]:
import pyspark.sql.functions as f
from pyspark.sql.functions import desc

df.groupBy("year_written").agg({"Price" : "max"}).sort(desc("year_written")).show()

+------------+----------+
|year_written|max(Price)|
+------------+----------+
|        2000|     19.95|
|        1999|     12.35|
|        1967|      14.0|
|        1937|     27.45|
|        1925|      25.0|
|        1922|      29.0|
|        1875|      13.5|
|        1870|      5.75|
|        1865|      12.7|
|        1862|      7.75|
|        1814|      18.2|
|        1603|      7.95|
+------------+----------+



* Tampilkan data buku termurah tiap tahun penulisannya!

In [26]:
import pyspark.sql.functions as f
from pyspark.sql.functions import desc

df.groupBy("year_written").agg({"Price" : "min"}).sort(desc("year_written")).show()

+------------+----------+
|year_written|min(Price)|
+------------+----------+
|        2000|     19.95|
|        1999|     12.35|
|        1967|      14.0|
|        1937|     27.45|
|        1925|      25.0|
|        1922|      29.0|
|        1875|      13.5|
|        1870|      5.75|
|        1865|      5.76|
|        1862|      7.75|
|        1814|      18.2|
|        1603|      7.95|
+------------+----------+

